MOUNTING DRIVE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive



OpenCV INSTALLATION:

In [ ]:
!pip install opencv-python

FRAME EXTRACTION:

In [ ]:
import cv2
import os

# Loading the  video file
video_path = '/content/drive/MyDrive/Intern/London Underground Extravaganza All 11 Lines! 29 November 2016.mp4'
cap = cv2.VideoCapture(video_path)

# the folder where frames will be stored
frames_folder = '/content/drive/MyDrive/Intern/frames'

# Frame extraction at one frame per second (FPS)
frame_count = 0
success = True

while success:
  success, frame = cap.read()
  if not success:
    break
  # Extracting a frame every 30 frames (1 FPS)
  if frame_count % 30 == 0:
    frame_path = os.path.join(frames_folder, f"frame_{frame_count}.jpg")
    cv2.imwrite(frame_path, frame)

  frame_count += 1

cap.release()



In [ ]:
print("Reading frame:", frame_count)
print("Saving frame:", frame_path)

Reading frame: 42438
Saving frame: /content/drive/MyDrive/Intern/frames/frame_42420.jpg


FACE RECOGNITION LIBRARY INSTALLATION:

In [ ]:
!pip install face_recognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566170 sha256=c04aa176a1deae49f910ae4de176d34d12a2c31fcc81a47968e08df9613d1da5
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


FACE RECOGNITON:

In [ ]:
import os
import cv2
import face_recognition
from collections import Counter

# Path of the folder which containing the  extracted frames
frames_folder = "/content/drive/MyDrive/Intern/frames"

# Load Haar cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Initializing variables for face recognition
known_face_encodings = []
known_face_names = []

# Process each frame
for frame_filename in os.listdir(frames_folder):
    frame_path = os.path.join(frames_folder, frame_filename)

    # Loading the image
    image = cv2.imread(frame_path)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detecting faces using Haar cascade
    faces_haar = face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Iterating over each detected face using Haar cascade
    for (x, y, w, h) in faces_haar:
        # Extracting face ROI
        face_roi = gray_image[y:y+h, x:x+w]

        # Identifying faces using face_recognition
        face_encoding = face_recognition.face_encodings(image, [(y, x+w, y+h, x)])[0]

        # Check if the detected face matches with known faces
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        name = "Unknown"

        # Check if any known face matches
        if any(matches):
            name = known_face_names[matches.index(True)]
        else:
            # If no match found, add the face to known faces
            known_face_encodings.append(face_encoding)
            known_face_names.append(f"Person_{len(known_face_names)+1}")

# Counting the occurrences of each unique face
face_counter = Counter(known_face_names)

# Determine the most occurring face in the complete video
most_common_face = face_counter.most_common(1)

print("\nMost occurring face in the complete video:")
if most_common_face:
    print(f" {most_common_face[0][0]}, Occurrences: {most_common_face[0][1]}")
else:
    print("No faces detected in the video.")
